# size distribution example
Load basic packages

In [1]:
import size_distribution_class as sdc
import statistics

import pandas as pd
import pandas_bokeh
import numpy as np
from pylab import *
from scipy.optimize import curve_fit
import scipy
import csv

from bokeh.io import output_file, output_notebook
from bokeh.models import DatetimeTickFormatter, ranges, Span, Label, Range1d, LinearAxis
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
from bokeh.palettes import linear_palette, Turbo256
from matplotlib import pyplot as plt
pd.set_option('plotting.backend', 'pandas_bokeh')
output_notebook(hide_banner=True)
from IPython.core.display import display, HTML



In [2]:
psd, cld_complete_histogram_df, psd_complete_histogram_df = sdc.Folder_Process( verbose=True, analyze_color_name='white', analyze_color_BGR=[0,0,255] )

Please select the directory with the source image tiles.
Found 8 files...
File 1 of 8
--------------------------------------------------
Analysing C3S 28d CryoBIB_002_pores.tif
size: 1243.33 x 1073.35 nm / 2048 x 1768 px 
loaded psd csv
--------------------------------------------------
loaded cld csv
File 2 of 8
--------------------------------------------------
Analysing C3S 28d CryoBIB_005_pores.tif
size: 1243.33 x 1073.35 nm / 2048 x 1768 px 
loaded psd csv
--------------------------------------------------
loaded cld csv
File 3 of 8
--------------------------------------------------
Analysing C3S 28d CryoBIB_009_pores.tif
size: 1243.33 x 1073.35 nm / 2048 x 1768 px 
loaded psd csv
--------------------------------------------------
loaded cld csv
File 4 of 8
--------------------------------------------------
Analysing C3S 28d CryoBIB_011_pores.tif
size: 1243.33 x 1073.35 nm / 2048 x 1768 px 
loaded psd csv
--------------------------------------------------
loaded cld csv
File 5 of 

In [18]:
def compress_histogram_bins( hist, bins, compression ):
    bins_compressed = []
    hist_compressed = []
    for i in range(int(len(bins)/compression)):
        value = 0
        for j in range(compression):
            value += hist[i*compression+j]
        hist_compressed.append( value )

    return hist_compressed, bins_compressed

def compress_histogram( hist, compression ):
    hist_compressed = []
    for i in range(int(len(hist)/compression)):
        value = 0
        for j in range(compression):
            #try:
            value += hist[i*compression+j]
            #except:
            #    print(len(hist), int(len(hist)/compression), i, j, hist)
        hist_compressed.append( value )

    return hist_compressed

def compress_bins( bins, compression ):
    bins_compressed = []
    for i in range(int(len(bins)/compression)):
        bins_compressed.append( bins[i*compression])

    return bins_compressed


compression = 1

data = []
columns = cld_complete_histogram_df.columns
compressed_bin = list(map(float, compress_bins(columns, compression)))
for index, row in cld_complete_histogram_df.fillna(0).iterrows():
    data.append( compress_histogram(row, compression) )
    
cld_complete_histogram_mean   = np.mean(data, axis=0)
cld_complete_histogram_median = np.median(data, axis=0)
cld_complete_histogram_std    = np.std(data, axis=0)
cld_complete_histogram_sum    = np.sum(data, axis=0)
#df = pd.DataFrame(data)
#df.to_csv('./b.csv')

print(cld_complete_histogram_mean)

compressed_bin =np.array( psd.getPoreArea( np.array(compressed_bin) ) )
cld_complete_histogram_mean = cld_complete_histogram_mean * compressed_bin
cld_complete_histogram_std = cld_complete_histogram_std * compressed_bin
print(cld_complete_histogram_mean)


cld_complete_histogram_mean = cld_complete_histogram_mean[:-1]
cld_complete_histogram_median = cld_complete_histogram_median[:-1]
cld_complete_histogram_std = cld_complete_histogram_std[:-1]
print(len(cld_complete_histogram_mean))
#print(cld_complete_histogram_sum)
#print(cld_complete_histogram_mean)
#print(cld_complete_histogram_median)
#print(cld_complete_histogram_std)
print(len(compressed_bin))


[ 0.    0.   10.25 ...  0.    0.    0.  ]
[ 0.          0.         11.86845545 ...  0.          0.
  0.        ]
2049
2050


In [19]:

"""
cld_complete_histogram_mean = []
cld_complete_histogram_median = []
cld_complete_histogram_std = []

for i in columns:
    cld_complete_histogram_mean.append(cld_complete_histogram_df[i].mean())
    cld_complete_histogram_median.append(cld_complete_histogram_df[i].median())
    cld_complete_histogram_std.append(cld_complete_histogram_df[i].std())

cld_complete_histogram_mean = np.array(cld_complete_histogram_mean[:-1])
cld_complete_histogram_median = np.array(cld_complete_histogram_median[:-1])
cld_complete_histogram_std = np.array(cld_complete_histogram_std[:-1])

columns = list(map(float, cld_complete_histogram_df.columns))
cld_complete_histogram_mean = cld_complete_histogram_mean * columns[:-1]
cld_complete_histogram_std = cld_complete_histogram_std * columns[:-1]

#cld_complete_histogram_mean, _ = compress_histogram_bins( cld_complete_histogram_mean, columns, 3 )
#cld_complete_histogram_median, columns = compress_histogram_bins( cld_complete_histogram_mean, columns, 3 )
"""

'\ncld_complete_histogram_mean = []\ncld_complete_histogram_median = []\ncld_complete_histogram_std = []\n\nfor i in columns:\n    cld_complete_histogram_mean.append(cld_complete_histogram_df[i].mean())\n    cld_complete_histogram_median.append(cld_complete_histogram_df[i].median())\n    cld_complete_histogram_std.append(cld_complete_histogram_df[i].std())\n\ncld_complete_histogram_mean = np.array(cld_complete_histogram_mean[:-1])\ncld_complete_histogram_median = np.array(cld_complete_histogram_median[:-1])\ncld_complete_histogram_std = np.array(cld_complete_histogram_std[:-1])\n\ncolumns = list(map(float, cld_complete_histogram_df.columns))\ncld_complete_histogram_mean = cld_complete_histogram_mean * columns[:-1]\ncld_complete_histogram_std = cld_complete_histogram_std * columns[:-1]\n\n#cld_complete_histogram_mean, _ = compress_histogram_bins( cld_complete_histogram_mean, columns, 3 )\n#cld_complete_histogram_median, columns = compress_histogram_bins( cld_complete_histogram_mean, col

In [20]:
def get_X_List( values ):
    value_cnt = len(values)
    x_range = list(range(value_cnt))
    x_range.pop(0)
    x_range.append(value_cnt)
    return x_range

def get_x(x, c):
    c_len = len(c)
    if c_len == 1:
        return c[0]
    else:
        return get_x(x, c[:-1])*x + c[c_len-1]

def get_polyfit_xy( x, y, deg=3, resolution=1 ):
    c = np.polynomial.polynomial.polyfit(x, y, deg)

    x_result = np.linspace(x[0], x[-1], num=len(x)*10)
    y_result = np.polynomial.polynomial.polyval(x_result, c)
    return x_result,y_result

def extrapolation_function(x, a, b, c):
    return a * np.exp(-b * x) + c

# see https://en.wikipedia.org/wiki/Inverse_Gaussian_distribution
def inv_gaussian_dist(x, lamb, mu, c=1.0):
    # since this causes problems with the curve_fit function and x is always larger than 0 in this case, the following two lines are not needed
    #if x <= 0.0:
    #    return 0.0              
    return c * np.sqrt( lamb / ( 2.0 * np.pi * x**3 )) * np.exp( ( -lamb * (x - mu)**2 ) / ( 2 * mu**2 * x ) )

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.fatiguelife.html#scipy.stats.fatiguelife
def fatiguelife(x, c):
    return scipy.stats.fatiguelife.pdf(x, c)#, loc, scale

def pareto(x,c):
    return scipy.stats.pareto.pdf(x, c)#, loc, scale

def powerlognorm(x, c, s):
    return scipy.stats.powerlognorm.pdf(x, c, s)

def get_histogram(dist_type='cld', x_column='diameter', y_column='count', max_value=None, plot_width=1200, legend_location='top_right', smoothing=0, do_fit=False, show_sum_list=False):
    # some basic label settings
    if dist_type == 'psd':
    #    df = psd.psd_df
        dist_name = 'pore size distribution'
        x_min_f=1
    else:
    #    df = psd.cld_df
        dist_name = 'chord length distribution'
    
    unit, power = psd.get_column_unit( x_column )


    # get & process the actual histogram
    #hist = cld_complete_histogram_mean #list(map(int, cld_complete_histogram_mean))
    #hist = cld_complete_histogram_median #list(map(int, cld_complete_histogram_mean))

    #hist = np.nan_to_num( cld_complete_histogram_std / cld_complete_histogram_median )*100
    hist = np.nan_to_num( cld_complete_histogram_std / cld_complete_histogram_mean )*100
    #bins = columns

    #hist = cld_complete_histogram_mean
    bins = compressed_bin
    #hist, bins = psd.get_cleaned_histogram_list(dist_type, x_column, y_column)
    #hist, bins = get_histogram_list(dist_type, x_column, y_column)
    x_min = bins[0]
    #print(max_value)
    #print(bins[-1])
    if max_value == None: max_value = bins[-1]
    x_max = bins[-1] if max_value >= bins[-1] else max_value
    
    title = 'blubb'#'{} as {} of {} cld_complete_histogram_median, x_column, psd.filename, max_value, unit'
    x_label = 'pore size ({}) in {}'.format(x_column, unit)
    y_label = 'amount of items in %'

    if x_column=='area':
        x_axis_type='log'
        x_axis_type='linear'
    elif x_column=='volume':
        x_axis_type='log'
    elif x_column=='surface':
        x_axis_type='log'
    else:
        x_axis_type='linear'

    # get the center of the bins
    bin_center = sdc.get_bin_center(bins)

    # normalize the histograms
    normalized_hist = hist#/(np.sum(hist*bin_center)/100)
    y_max = np.amax(normalized_hist)
    
    first_val_pos = 0
    
    # create the figure
    p = figure(title=title,  plot_width=plot_width, x_range=(x_min, x_max), y_range=(0,y_max), plot_height=600, background_fill_color="#fafafa", x_axis_type=x_axis_type)
    p.legend.location = legend_location
    p.xaxis.axis_label = x_label

    # histogram of raw data
    p.quad(bottom=0, top=hist, left=bins[1:], right=bins[:-1], fill_color="blue", line_color="blue")

    # smoothed raw data
    if False:#smoothing > 2:
        hist_smooth = sdc.smooth_histogram(normalized_hist, window_len=smoothing)
        p.line(bin_center, hist_smooth, line_color="#ff88ff", line_width=4, alpha=0.7, legend_label="smoothed")

    #polynome fit
    if False:#do_fit:
        # fit to a predefined function
        # https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html  lamb, mu, c
        
        first_val_y = bin_center[first_val_pos]
        popt, pcov = curve_fit(powerlognorm, bin_center[first_val_pos:], normalized_hist[first_val_pos:], bounds=([0, -np.inf], np.inf))
        
        label='-'
        x_fit = np.linspace(int(bin_center[first_val_pos:][0]), int(bin_center[-1])+1, num=len(bin_center)*2)
        y_fit = powerlognorm(x_fit-first_val_y, *popt)
        p.line(x_fit, y_fit, line_color="#11aa11", line_width=2, alpha=0.7, legend_label=label)

    if False:#show_sum_list:
        # iterated sum
        sum_list = psd.get_sum_list(normalized_hist)
        p.extra_y_ranges = {"sum_line": Range1d(start=0, end=np.amax(sum_list))}

        p.line(bins, sum_list, line_color="#ff8888", line_width=4, alpha=0.7, legend_label="Sum", y_range_name="sum_line")
        p.add_layout(LinearAxis(y_range_name="sum_line",axis_label=y_label), 'left')

    
    show(p)

In [21]:
cld_complete_histogram_df.columns

Index(['0.0', '0.6071', '1.2142', '1.8213', '2.4284', '3.0355', '3.6426',
       '4.2497', '4.8568', '5.4639',
       ...
       '1238.4758', '1239.0829', '1239.69', '1240.2971', '1240.9042',
       '1241.5113', '1242.1184', '1242.7255', '1243.3326', '1243.9397'],
      dtype='object', length=2050)

generate Diagrams 

In [25]:
get_histogram('psd','area', show_sum_list=False, max_value = 50)

In [23]:

histo = np.nan_to_num( cld_complete_histogram_std / cld_complete_histogram_mean )*100
print(len(histo))
print(max(histo))

2049
264.57513110645914


In [24]:
print(max(cld_complete_histogram_mean))
print(cld_complete_histogram_mean[-1])

30910.35039592055
0.0
